#Variant Filtration Pipeline for FVL_SUP project


##Fastq files

All generated fastq files have been deposited to the [NCBI Sequence Read Archive](http://www.ncbi.nlm.nih.gov/sra) (project accession number \#PRJNA296481)

\#PRJNA296481 encompasses 32 whole exome sequencing experiments from 32 different mice representing 8 ENU suppressor lines: *MF5L*1, *MF5L*5, *MF5L*6, *MF5L*8, *MF5L*9, *MF5L*11, *MF5L*12, *MF5L*16.

Current pipeline calls variants jointly from all 32 sequencing experiments, and then focuses on 8 mice representing each of the ENU suppressor lines: \#SRR2473338, \#SRR2473339, \#SRR2473340, \#SRR2473342, \#SRR2473343, \#SRR2473344, \#SRR2473347, \#SRR2473348. Details below!

##Overview of variant calling pipeline

Reference genome: Mus_musculus GRCm38 release 73, downloaded from Ensemble

Used software: 
* bwa (version 0.7.5a-r405)
* picard-tools (version 1.105)
* GenomeAnalysisTK (version 2.6.5)

Steps to obtain the .vcf files:
1. bwa aln -q 15
 bwa sampe
2. picard-tools/SortSam.jar SORT_ORDER=coordinate VALIDATION_STRINGENCY=SILENT CREATE_INDEX=true
3. picard-tools/MarkDuplicates.jar VALIDATION_STRINGENCY=SILENT REMOVE_DUPLICATES=true ASSUME_SORTED=true CREATE_INDEX=true
4. GenomeAnalysisTK/GenomeAnalysisTK.jar -T RealignerTargetCreator
5. GenomeAnalysisTK/GenomeAnalysisTK.jar -T IndelRealigner
6. picard-tools/FixMateInformation.jar SORT_ORDER=coordinate VALIDATION_STRINGENCY=SILENT CREATE_INDEX=true
7. GenomeAnalysisTK/GenomeAnalysisTK.jar -T HaplotypeCaller -stand_call_conf 50.0 stand_emit_conf
8. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantAnnotator -A VariantType
9. GenomeAnalysisTK/GenomeAnalysisTK.jar -T SelectVariants -selectType SNP 
10. GenomeAnalysisTK/GenomeAnalysisTK.jar -T SelectVariants -selectType INDEL
11. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantFiltration --variant SNP.vcf --filterExpression 'QD < 2.0 || MQ < 40.0 || FS > 60.0 || HaplotypeScore > 13.0 || MQRankSum < -12.5 || ReadPosRankSum < -8.0' --filterName 'FAIL'
12. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantFiltration --variant INDEL.vcf --filterExpression "QD < 2.0 || FS > 200.0 || ReadPosRankSum < -20.0 || InbreedingCoeff < -0.8" --filterName FAILED

_End result is two vcf files: SNP.vcf and INDEL.vcf_

#Preparation and Annotation of called SNP.vcf and INDEL.vcf

Details on Annovar version and reference sequence:

1. $LastChangedDate: 2012-05-25
2. annotate_variation.pl --buildver mm10 --downdb seq mousedb/mm10_seq
_Downloading annotation database ftp://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/chromFa.tar.gz ... OK_

In [ ]:
#UNIX commands:
    
grep '#CHROM' SNP.vcf > header.txt
cat SNP.vcf INDEL.vcf | sort -k1n -k2n | grep -v '#' > merged.vcf

convert2annovar.pl merged.vcf -format vcf4old -includeinfo > merged
annotate_variation.pl merged --buildver mm10 /database/annovar/mousedb/

#Variant Filtration Pipeline

Input files are Annovar output files 'merged.exonic_variant_function' and 'merged.variant_function'

In [2]:
#import all necessary packages
import sys
import numpy as np
import re
import vcf
import subprocess
from __future__ import print_function

In [2]:
def reformatAnnovar(FileName,Header):
    '''reformatAnnovar will merge .exonic_variant_function and .variant_function
    back into one vcf file while adding Annovar Annotation into the INFO column'''
    
    input1=open(FileName+'.variant_function','r')
    input2=open(FileName+'.exonic_variant_function','r')
    outfile=open(FileName+'_annot.vcf','w')
    header=open(Header,'r').readline().rstrip()
    rows=[]
    exonic=dict()
    
    print(header,file=outfile)
    
    for line in input2:
        line=line.strip()
        row=line.split('\t')
        row[15]=row[15]+';AnnovarType='+'_'.join(row[1].split())+';AnnovarGene='+'_'.join(row[2].split())
        new_line=row[8:len(row)]        
        new_line='\t'.join(new_line)
        exonic[row[3]+'_'+row[4]]=new_line

    for line in input1:
        line=line.strip()
        row=line.split('\t')
        
        if row[2]+'_'+row[3] in exonic:
            print(exonic[row[2]+'_'+row[3]],file=outfile)
            continue
        else:
            row[14]=row[14]+';AnnovarType='+'_'.join(row[0].split())+';AnnovarGene='+'_'.join(row[1].split())
            new_line=row[7:len(row)]
            print('\t'.join(new_line),file=outfile)
            continue

In [3]:
def VariantDictionaryFromVcf(VCFfile):
    '''This Function Takes the variants in a given VCF file
    and returns them in a dictionary formatted as chr_pos as keys'''
    input=open(VCFfile)
    variants=dict()
    for line in input:
        if not line.startswith('#'):
            row=line.rstrip().split('\t')
            variants[row[0]+'_'+row[1]]=0
    
    return variants        

Here, I merge Annovar annotation files of the original 'merged.vcf' file into **'merged_annot.vcf'** file. 'merged_annot.vcf' file will be used for the downstream filtering.

In [ ]:
reformatAnnovar('merged','header.txt')

Before filtering, I am creating a dictionary of all known C57BL/6J and 129/SvImJ SNPs necessary for one of the filtering steps! The .vcf files for these SNPs and INDELs (GRCm38) is downloaded from the [Mouse Genomes Project](http://www.sanger.ac.uk/resources/mouse/genomes/).

In [7]:
IMJ=VariantDictionaryFromVcf('../../../NBEAL2/IPYTHON/129_mm10_variants.vcf')

Before filtering the variants, I am extracting data for the 8 mice we are interested in this project and reorder them according to the MF5L lines:

|MouseID|SRA Accession|Name in VCF|ENU Line|
|-|-|-|-|
|37201|\#SRR2473340|214_461_47|MF5L1|
|33902|\#SRR2473347|214_461_52|MF5L5|
|33877|\#SRR2473338|214_461_46|MF5L6|
|33590|\#SRR2473344|214_461_51|MF5L8|
|31424|\#SRR2473343|214_461_50|MF5L9|
|33501|\#SRR2473342|214_461_49|MF5L11|
|33460|\#SRR2473340|214_461_48|MF5L12|
|39044|\#SRR2473348|214_461_52|MF5L16|


In [ ]:
#UNIX command:
awk '{OFS="\t";print $1,$2,$3,$4,$5,$6,$7,$8,$9,$35,$40,$34,$39,$38,$37,$36,$41}' merged_annot.vcf > leiden8.vcf

**FILTER 1** from 'leiden8.vcf' to 'leiden8_filter1.vcf'

This step will apply multiple quality filters to identify likely ENU induced mutations in the 8 different suppressor line mice. This includes the following filters:
- variant has to be in heterozygous state
- alt allele unique to each mouse compared to other sequenced suppressor mice
- variant called in at least 50% of the mice (6) to be able to assess uniqueness
- heterozygous call has to have at least 6 reads and an allele ratio between 1/2 and 2 for reads DB<=10 and between 1/3 and 3 for DB>10
- variant does not overlap C57BL/6J and 129/SvImJ known SNPs

output files are going to be separate for each suppressor line

_Count of all variants entering this step, of variants removed by each filter, and of variants that passed all filters are printed to stdout_

In [104]:
vcfFile = vcf.Reader( open('leiden8.vcf','r') )
MF5L1 = vcf.Writer(open('MF5L1_filter1.vcf', 'w'), vcfFile)
MF5L5 = vcf.Writer(open('MF5L5_filter1.vcf', 'w'), vcfFile)
MF5L6 = vcf.Writer(open('MF5L6_filter1.vcf', 'w'), vcfFile)
MF5L9 = vcf.Writer(open('MF5L9_filter1.vcf', 'w'), vcfFile)
MF5L10 = vcf.Writer(open('MF5L10_filter1.vcf', 'w'), vcfFile)
MF5L12 = vcf.Writer(open('MF5L12_filter1.vcf', 'w'), vcfFile)
MF5L13 = vcf.Writer(open('MF5L13_filter1.vcf', 'w'), vcfFile)
MF5L22 = vcf.Writer(open('MF5L22_filter1.vcf', 'w'), vcfFile)

printIDs=[MF5L1,MF5L5,MF5L6,MF5L9,MF5L10,MF5L12,MF5L13,MF5L22]
miceIDs=['214_461_47','214_461_52','214_461_46',
         '214_461_51','214_461_50','214_461_49','214_461_48','214_461_53']

#filters statistics
counter=0
(f1,f2,f3,f4,f5,passed)=(0,0,0,0,0,0)
p=[0]*8

for record in vcfFile:  
    counter+=1    
    
    #remove variants that did not pass the GATK filters:
    if not len(record.FILTER)==0:
        f1+=1
        continue
    
    #remove variants where there are more alleles as hets or homozygous calls:
    if record.num_het > 1 or record.num_hom_alt > 0:
        f2+=1
        continue
              
    #remove variants where 50% of the mice (4 mice) have no genotype called      
    if record.num_unknown>=4:
        f3+=1
        continue 

    #called '0/1' variant should have read depth of 6
    #with allele ratios between 1/3 and 3 (1/2 and 2 for DP<=10):
    
    count_DP=0
    for i in miceIDs:
        if not record.genotype(i)==None and not record.genotype(i)['GT']==None:
            if not record.genotype(i)['GT']=='0/1':
                continue
            if not hasattr(record.genotype(i).data,'DP'):
                continue
            if not record.genotype(i)['DP']==None: 
                if int(record.genotype(i)['DP'])>=6:
                    ref=float(record.genotype(i)['AD'][0])
                    alt=float(record.genotype(i)['AD'][1])
                    
                    if alt==0 or ref==0:
                        continue 
                        
                    dev=alt/ref
                    
                    if int(record.genotype(i)['DP'])<=10:
                        if dev<0.5 or dev>2:
                            continue
                    if int(record.genotype(i)['DP'])>10:
                        if dev<0.333 or dev>3:
                            continue
                    count_DP+=1
                
    if count_DP==0:
        f4+=1
        continue
    
    #remove all variants overlapping with known C57BL/6J and 129S1/SvIMJ SNPs 
    if str(record.CHROM)+'_'+str(record.POS) in IMJ:
        f5+=1
        continue
        
    passed+=1

    #print out mice to their corresponding files
    for i in miceIDs:
        if not record.genotype(i)==None and not record.genotype(i)['GT']==None:
            if record.genotype(i)['GT']=='0/1':
                outfile=printIDs[miceIDs.index(i)]
                outfile.write_record(record)
                p[miceIDs.index(i)]+=1
            else:
                continue

for i in printIDs:
    i.close()
    
print('all','GATK','shared','<50%GT','badDP','129var','passed',sep='\t')   
print(counter,f1,f2,f3,f4,f5,passed,sep='\t')
print('MF5L1','MF5L5','MF5L6','MF5L9','MF5L10','MF5L12','MF5L13','MF5L22',sep='\t')
print('\t'.join(map(str,p)))

all	GATK	shared	<50%GT	badDP	129var	passed
7012855	1598268	5128174	204821	78116	944	2532
MF5L1	MF5L5	MF5L6	MF5L9	MF5L10	MF5L12	MF5L13	MF5L22
291	164	393	597	288	232	240	327


**FILTER 2** removes variants that are <=200bp apart within each mouse:

In [3]:
def RemoveVariantsWithin200bp(FileName,OutFile):
    '''Removes variants that are closer than 200 bp to each other'''
    
    input=open(FileName)
    outfile=open(OutFile,'w')
    
    header=input.readline().rstrip()
    print(header,file=outfile)

    chr=''
    pos=0
    removed_pos=0
    rows=[]
    counter=0
    passed=0

    for line in input:
        counter+=1
        line=line.rstrip()
        row=line.split('\t')
        if not row[0]==chr:
            chr=row[0]
            pos=int(row[1])
            rows.append(line)
            removed_pos=0
            continue
        else:
            if int(row[1])-removed_pos <= 200:
                    removed_pos=int(row[1])
                    continue
            else:
                if int(row[1])-pos <= 200:
                    rows.pop()
                    removed_pos=int(row[1])
                    continue
                else:
                    pos=int(row[1])
                    rows.append(line)
                    continue

    for line in rows:
        passed+=1
        print(line,file=outfile)

    #for stats, this is the number discarded  variants
    print('all','passed',sep='\t')   
    print(counter,passed,sep='\t')
    outfile.close()

In [4]:
RemoveVariantsWithin200bp('MF5L1_filter1.vcf','MF5L1_filter2.vcf')
RemoveVariantsWithin200bp('MF5L5_filter1.vcf','MF5L5_filter2.vcf')
RemoveVariantsWithin200bp('MF5L6_filter1.vcf','MF5L6_filter2.vcf')
RemoveVariantsWithin200bp('MF5L9_filter1.vcf','MF5L9_filter2.vcf')
RemoveVariantsWithin200bp('MF5L10_filter1.vcf','MF5L10_filter2.vcf')
RemoveVariantsWithin200bp('MF5L12_filter1.vcf','MF5L12_filter2.vcf')
RemoveVariantsWithin200bp('MF5L13_filter1.vcf','MF5L13_filter2.vcf')
RemoveVariantsWithin200bp('MF5L22_filter1.vcf','MF5L22_filter2.vcf')

all	passed
291	200
all	passed
164	152
all	passed
393	293
all	passed
597	439
all	passed
288	231
all	passed
232	218
all	passed
240	188
all	passed
327	274


In [5]:
def FilterCodingVariants(FileName,OutFile):
    '''This generator will use the Annovar annotation done above to filter
    out variants within coding region and display them on screen'''

    input=open(FileName)
    outfile=open(OutFile,'w')
    types={'nonsynonymous_SNV':0,'stopgain':0,
          'stoploss':0,'splicing':0}
    counter=0
    passed=0

    for line in input:
        counter+=1
        type=''
        OK=0
        line=line.rstrip()
        row=line.split('\t')
        info=row[7].split(';')
        for i in info:
            if i.startswith('AnnovarGene'):
                gene=i.split('=')[1].split(':')[0]
            if i.startswith('AnnovarType'):
                type=i.split('=')[1]
                if type in types:
                    OK=1
                    passed+=1
                else:
                    continue
            else:
                continue
            if OK==1:    
                print(line,file=outfile)
                print(row[0],row[1],row[3],row[4],row[9].split(':')[1],row[10].split(':')[1],row[11].split(':')[1], type,gene,sep='\t')

    print('\nall','passed',sep='\t')   
    print(counter,passed,'\n',sep='\t')
    outfile.close()  

In [6]:
FilterCodingVariants('MF5L1_filter2.vcf','MF5L1_filter3.vcf')
FilterCodingVariants('MF5L5_filter2.vcf','MF5L5_filter3.vcf')
FilterCodingVariants('MF5L6_filter2.vcf','MF5L6_filter3.vcf')
FilterCodingVariants('MF5L9_filter2.vcf','MF5L9_filter3.vcf')
FilterCodingVariants('MF5L10_filter2.vcf','MF5L10_filter3.vcf')
FilterCodingVariants('MF5L12_filter2.vcf','MF5L12_filter3.vcf')
FilterCodingVariants('MF5L13_filter2.vcf','MF5L13_filter3.vcf')
FilterCodingVariants('MF5L22_filter2.vcf','MF5L22_filter3.vcf')

2	70509665	G	C	31,23	81,0	81,0	nonsynonymous_SNV	Erich2
7	14225894	T	C	94,46	114,0	181,28	nonsynonymous_SNV	Sult2a6
7	15940142	T	C	7,10	32,0	29,0	nonsynonymous_SNV	Gltscr2
7	85754889	A	T	81,36	45,0	138,25	nonsynonymous_SNV	Vmn2r72
8	108949251	A	G	55,20	59,0	47,0	nonsynonymous_SNV	Zfhx3
12	110977486	G	A	5,6	46,0	38,0	nonsynonymous_SNV	Ankrd9
17	25722876	T	A	37,40	83,0	91,0	nonsynonymous_SNV	Chtf18

all	passed
201	7	

3	99352190	A	G	143,0	89,109	152,0	nonsynonymous_SNV	Tbx15
3	135228816	T	A	18,0	7,8	20,0	nonsynonymous_SNV	Cenpe
6	35080128	G	A	46,0	23,18	55,0	nonsynonymous_SNV	Cnot4
8	70913530	A	T	52,0	67,43	121,0	nonsynonymous_SNV	Map1s
17	70657633	T	G	35,0	11,22	21,0	splicing	Dlgap1(NM_027712

all	passed
153	5	

1	82741945	C	A	106,1	127,2	34,56	nonsynonymous_SNV	Mff
2	67516594	A	G	137,0	141,0	92,80	nonsynonymous_SNV	Xirp2
2	76724952	G	A	35,0	47,0	27,20	nonsynonymous_SNV	Ttn
2	76939280	T	A	58,0	56,0	41,29	nonsynonymous_SNV	Ttn
2	88423385	A	T	114,0	101,0	74,65	nonsynonymous_SNV	Olfr1181
2